<a href="https://colab.research.google.com/github/donbcolab/composable_vlms/blob/main/notebooks/roboflow_bccd_hf_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Roboflow 100 - Blood Cell Dataset (BCCD)

In [1]:
!pip install -q roboflow datasets datasets[vision]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is i

## Setup Roboflow connection parameters and configuration BCCD dataset

In [2]:
from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("roboflow-100").project("bccd-ouzjz")
version = project.version(2)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /content/coco/ in coco:: 100%|██████████| 372/372 [00:00<00:00, 6072.69it/s]


Extracting Dataset Version Zip to /content/paligemma/ in paligemma:: 100%|██████████| 370/370 [00:00<00:00, 5702.03it/s]


Extracting Dataset Version Zip to /content/florence2_od/ in florence2-od:: 100%|██████████| 372/372 [00:00<00:00, 4983.98it/s]


### select dataset model formats - for models you want to finetune

In [ ]:
dataset = version.download(model_format="paligemma", location="/content/paligemma/", overwrite=True)
dataset = version.download(model_format="florence2-od", location="/content/florence2_od/", overwrite=True)

### validation

In [3]:
# for each of the dataset model formats (florence2_od / paligemma)
# do a "ls -l" of the json files
!ls -l florence2_od/test/*.json*
!ls -l florence2_od/train/*.json*
!ls -l florence2_od/valid/*.json*

!ls -l paligemma/dataset/*.json*

-rw-r--r-- 1 root root 1122675 Jul 23 03:30 coco/train/_annotations.coco.json
-rw-r--r-- 1 root root 153472 Jul 23 03:30 coco/test/_annotations.coco.json
-rw-r--r-- 1 root root 314313 Jul 23 03:30 coco/valid/_annotations.coco.json
-rw-r--r-- 1 root root 21993 Jul 23 03:30 florence2_od/test/annotations.jsonl
-rw-r--r-- 1 root root 160106 Jul 23 03:30 florence2_od/train/annotations.jsonl
-rw-r--r-- 1 root root 45081 Jul 23 03:30 florence2_od/valid/annotations.jsonl
-rw-r--r-- 1 root root  24860 Jul 23 03:30 paligemma/dataset/_annotations.test.jsonl
-rw-r--r-- 1 root root 180953 Jul 23 03:30 paligemma/dataset/_annotations.train.jsonl
-rw-r--r-- 1 root root  50942 Jul 23 03:30 paligemma/dataset/_annotations.valid.jsonl


In [4]:
import os

def count_jpg_files(directory):
    return len([f for f in os.listdir(directory) if f.endswith('.jpg')])

directories = [
    "/content/florence2_od/test",
    "/content/florence2_od/train",
    "/content/florence2_od/valid",
    "/content/paligemma/dataset"
]

for directory in directories:
    try:
        count = count_jpg_files(directory)
        print(f"{directory}: {count} files")
    except FileNotFoundError:
        print(f"{directory}: Directory not found")


/content/coco/train: 255 files
/content/coco/test: 36 files
/content/coco/valid: 73 files
/content/florence2_od/test: 36 files
/content/florence2_od/train: 255 files
/content/florence2_od/valid: 73 files
/content/paligemma/dataset: 364 files


## Florence 2 - dataset preparation

In [9]:
import os
import json
from datasets import Dataset, Features, Image, Value, DatasetDict

def load_florence2_annotations(annotation_file, image_dir):
    dataset_entries = []
    with open(annotation_file, 'r') as f:
        for line in f:
            ann = json.loads(line)
            entry = {
                'image_id': ann['image'],
                'image': os.path.join(image_dir, ann['image']),
                'annotations': ann  # Keep all original fields
            }
            dataset_entries.append(entry)
    return dataset_entries

# Load annotations
train_data = load_florence2_annotations('/content/florence2_od/train/annotations.jsonl', '/content/florence2_od/train')
test_data = load_florence2_annotations('/content/florence2_od/test/annotations.jsonl', '/content/florence2_od/test')
valid_data = load_florence2_annotations('/content/florence2_od/valid/annotations.jsonl', '/content/florence2_od/valid')

# Define features
features = Features({
    'image_id': Value('string'),
    'image': Image(),
    'annotations': {
        'image': Value('string'),
        'prefix': Value('string'),
        'suffix': Value('string')
    }
})

# Create datasets
florence2_train_dataset = Dataset.from_list(train_data, features=features)
florence2_test_dataset = Dataset.from_list(test_data, features=features)
florence2_valid_dataset = Dataset.from_list(valid_data, features=features)

# Combine into a DatasetDict
florence2_dataset = DatasetDict({
    'train': florence2_train_dataset,
    'test': florence2_test_dataset,
    'validation': florence2_valid_dataset
})

### push dataset to hub

In [10]:
# Push to Hub - uncomment next line when ready to push (update with your id)
# florence2_dataset.push_to_hub("dwb2023/roboflow100-bccd-florence2")

# Print some information about the dataset
print(f"Dataset splits: {florence2_dataset.keys()}")
print(f"Number of training examples: {len(florence2_dataset['train'])}")
print(f"Number of testing examples: {len(florence2_dataset['test'])}")
print(f"Number of validation examples: {len(florence2_dataset['validation'])}")
print(f"Features: {florence2_dataset['train'].features}")

# Print a sample entry
print("\nSample entry:")
print(florence2_dataset['train'][0])

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/255 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset splits: dict_keys(['train', 'test', 'validation'])
Number of training examples: 255
Number of testing examples: 36
Number of validation examples: 73
Features: {'image_id': Value(dtype='string', id=None), 'image': Image(mode=None, decode=True, id=None), 'annotations': {'image': Value(dtype='string', id=None), 'prefix': Value(dtype='string', id=None), 'suffix': Value(dtype='string', id=None)}}

Sample entry:
{'image_id': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7B72E44569E0>, 'annotations': {'image': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'prefix': '<OD>', 'suffix': 'RBC<loc_756><loc_406><loc_958><loc_631>RBC<loc_820><loc_623><loc_995><loc_825>RBC<loc_150><loc_327><loc_355><loc_508>RBC<loc_283><loc_685><loc_283><loc_685>RBC<loc_400><loc_417><loc_588><loc_640>RBC<loc_817><loc_2><loc_999><loc_248>RBC<loc_50><loc_10><loc_208><loc_204>RBC<loc_206><loc_46><loc_3

## Paligemma - dataset preparation

In [11]:
import os
import json
from datasets import Dataset, Features, Image, Value, DatasetDict

def load_paligemma_annotations(annotation_file, image_dir):
    dataset_entries = []
    with open(annotation_file, 'r') as f:
        for line in f:
            ann = json.loads(line)
            entry = {
                'image_id': ann['image'],
                'image': os.path.join(image_dir, ann['image']),
                'annotations': ann  # Keep all original fields
            }
            dataset_entries.append(entry)
    return dataset_entries

# Load annotations
train_data = load_paligemma_annotations('/content/paligemma/dataset/_annotations.train.jsonl', '/content/paligemma/dataset')
test_data = load_paligemma_annotations('/content/paligemma/dataset/_annotations.test.jsonl', '/content/paligemma/dataset')
valid_data = load_paligemma_annotations('/content/paligemma/dataset/_annotations.valid.jsonl', '/content/paligemma/dataset')

# Define features
features = Features({
    'image_id': Value('string'),
    'image': Image(),
    'annotations': {
        'image': Value('string'),
        'prefix': Value('string'),
        'suffix': Value('string')
    }
})

# Create datasets
paligemma_train_dataset = Dataset.from_list(train_data, features=features)
paligemma_test_dataset = Dataset.from_list(test_data, features=features)
paligemma_valid_dataset = Dataset.from_list(valid_data, features=features)

# Combine into a DatasetDict
paligemma_dataset = DatasetDict({
    'train': paligemma_train_dataset,
    'test': paligemma_test_dataset,
    'validation': paligemma_valid_dataset
})

### push dataset to hub

In [12]:
# Push to Hub - uncomment next line when ready to push (update with your id)
# paligemma_dataset.push_to_hub("dwb2023/roboflow-bccd-paligemma")

# Print some information about the dataset
print(f"Dataset splits: {paligemma_dataset.keys()}")
print(f"Number of training examples: {len(paligemma_dataset['train'])}")
print(f"Number of testing examples: {len(paligemma_dataset['test'])}")
print(f"Number of validation examples: {len(paligemma_dataset['validation'])}")
print(f"Features: {paligemma_dataset['train'].features}")

# Print a sample entry
print("\nSample entry:")
print(paligemma_dataset['train'][0])

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/255 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset splits: dict_keys(['train', 'test', 'validation'])
Number of training examples: 255
Number of testing examples: 36
Number of validation examples: 73
Features: {'image_id': Value(dtype='string', id=None), 'image': Image(mode=None, decode=True, id=None), 'annotations': {'image': Value(dtype='string', id=None), 'prefix': Value(dtype='string', id=None), 'suffix': Value(dtype='string', id=None)}}

Sample entry:
{'image_id': 'BloodImage_00327_jpg.rf.dc708dec2caf7249ddca49d41055ca4c.jpg', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7B72E44FFB20>, 'annotations': {'image': 'BloodImage_00327_jpg.rf.dc708dec2caf7249ddca49d41055ca4c.jpg', 'prefix': 'detect Platelets ; RBC ; WBC', 'suffix': '<loc0232><loc0169><loc0296><loc0233> Platelets ; <loc0217><loc0529><loc0302><loc0606> Platelets'}}


## Compare records across datasets

In [13]:
# Search for same image across datasets
image_id_to_find = 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg'

# Function to find a record by image_id
def find_record_by_image_id(dataset, image_id):
    for record in dataset['train']:
        if record['image_id'] == image_id:
            return record
    return None

In [15]:
# Find the Roboflow Florence2 record
record = find_record_by_image_id(florence2_dataset, image_id_to_find)

# Display the record
if record:
    print(f"Record found: {record}")
else:
    print("Record not found")

Record found: {'image_id': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7B72E4454BB0>, 'annotations': {'image': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'prefix': '<OD>', 'suffix': 'RBC<loc_756><loc_406><loc_958><loc_631>RBC<loc_820><loc_623><loc_995><loc_825>RBC<loc_150><loc_327><loc_355><loc_508>RBC<loc_283><loc_685><loc_283><loc_685>RBC<loc_400><loc_417><loc_588><loc_640>RBC<loc_817><loc_2><loc_999><loc_248>RBC<loc_50><loc_10><loc_208><loc_204>RBC<loc_206><loc_46><loc_314><loc_302>RBC<loc_528><loc_677><loc_670><loc_925>Platelets<loc_2><loc_752><loc_75><loc_854>Platelets<loc_109><loc_429><loc_184><loc_531>WBC<loc_286><loc_2><loc_664><loc_319>'}}


In [16]:
# Find the Roboflow Paligemma record
record = find_record_by_image_id(paligemma_dataset, image_id_to_find)

# Display the record
if record:
    print(f"Record found: {record}")
else:
    print("Record not found")

Record found: {'image_id': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7B72E4456E60>, 'annotations': {'image': 'BloodImage_00343_jpg.rf.d8c56063ce5e40c50efb00a7e0c83c3b.jpg', 'prefix': 'detect Platelets ; RBC ; WBC', 'suffix': '<loc0416><loc0774><loc0646><loc0980> RBC ; <loc0637><loc0840><loc0844><loc1019> RBC ; <loc0334><loc0153><loc0520><loc0363> RBC ; <loc0701><loc0289><loc0701><loc0289> RBC ; <loc0426><loc0409><loc0654><loc0601> RBC ; <loc0002><loc0836><loc0253><loc1023> RBC ; <loc0010><loc0051><loc0209><loc0212> RBC ; <loc0046><loc0211><loc0309><loc0321> RBC ; <loc0693><loc0540><loc0947><loc0686> RBC ; <loc0770><loc0001><loc0874><loc0076> Platelets ; <loc0439><loc0112><loc0544><loc0188> Platelets ; <loc0002><loc0292><loc0326><loc0680> WBC'}}
